In [2]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [12]:
coll_2017_df = pd.read_csv('D:/4142proj/Notebooks/Data/2017_Tabular_Transportation_Collision_Data.csv')[:50]

In [13]:
coll_2017_df.iloc[0]['Time']

'1899-12-31T19:48:00.000Z'

In [14]:
#Convert string to timestamp
time = datetime.strptime(coll_2017_df.iloc[0]['Time'], '%Y-%m-%dT%H:%M:%S.000Z')

In [17]:
collision_df = pd.DataFrame(columns=['Accident_Key','Accident_Time', 'Environment', 'Road_Surface', 'Traffic_Control',
                                     'Visibility', 'Impact_Type'])

location_df = pd.DataFrame(columns=['Location_Key','Street_Name', 'Intersection_1', 'Intersection_2', 'Longitude', 'Latitude'])


hour_df = pd.DataFrame(columns=['Hour_Key','Hour_Start', 'Hour_End', 'Date', 'Month', 'Year'])

for idx, row in coll_2017_df.iterrows():
    if idx!=4:
        continue
    #Accident
    accident_row = [idx + 100]
    
    time = datetime.strptime(row['Time'], '%Y-%m-%dT%H:%M:%S.000Z')
    accident_row.append(str(time.hour) + ":" + str(time.minute) + ":" + str(time.second))
    
    accident_row.append(row['Environment'][4:].strip())
    
    accident_row.append(row['Road_Surface'][4:].strip())
    
    accident_row.append(row['Traffic_Control'][4:].strip())
    
    accident_row.append(row['Light'][4:].strip())
    
    accident_row.append(row['Impact_type'][4:].strip())
    
    collision_df.loc[len(collision_df)] = accident_row
    print(accident_row)


[104, '8:20:0', 'Rain', 'Wet', 'Yield sign', 'Dark', 'Sideswipe']


In [ ]:
for idx, row in coll_2017_df.iterrows():    
    #Location
    
    location_row = [idx + 200]
    
    if '@' in row['Location']: 
        roads = row['Location'].split("@")
        location_row += [roads[0].strip(), roads[1].strip(), "NA"] #Street, Intersection 1, Intersection 2
        
    else: #when there is a btwn in the location
        roads = row['Location'].split("btwn")
        
        if "&" in roads[1]:
            intersections = roads[1].split("&")
            location_row += [roads[0].strip(), intersections[0].strip(), 
                             intersections[1].strip()] #Street, Intersection 1, Intersection 2
        else:
            location_row += [roads[0].strip(), roads[1].strip(), "NA"] #Street, Intersection 1, Intersection 2
        
    location_row += [row['Longitude'], row['Latitude']]
    
    location_df.loc[len(location_df)] = location_row

In [ ]:
for idx, row in coll_2017_df.iterrows():
    #Hour
    
    hour_row = [idx + 300]
    
    date = datetime.strptime(row['Date'], '%Y-%m-%dT%H:%M:%S.000Z')
    next_time = time + timedelta(hours=1)

    hour_row += [time.hour, next_time.hour]
    hour_row.append(str(date.day) + "/" + str(date.month) + "/" + str(date.year))
    hour_row += [date.month, date.year]
    
    hour_df.loc[len(hour_df)] = hour_row

In [16]:
collision_df.head()

,Accident_Key,Accident_Time,Environment,Road_Surface,Traffic_Control,Visibility,Impact_Type
0,100,19:48:0,Clear,Dry,Traffic signal,Daylight,Sideswipe
1,101,12:51:0,Snow,Slush,Traffic signal,Daylight,Rear end
2,102,18:31:0,Rain,Wet,Traffic signal,Daylight,Sideswipe
3,103,13:38:0,Clear,Ice,Traffic signal,Daylight,Rear end
4,104,8:20:0,Rain,Wet,Yield sign,Dark,Sideswipe


In [19]:
location_df.head()

,Location_Key,Street_Name,Intersection_1,Intersection_2,Longitude,Latitude
0,200,AIRPORT PKWY,HUNT CLUB RD,NA,-75.654907,45.350155
1,201,AIRPORT PKWY,HUNT CLUB RD,NA,-75.654900,45.350147
2,202,AIRPORT PKWY,HUNT CLUB RD,NA,-75.654907,45.350140
3,203,AIRPORT PKWY,HUNT CLUB RD,NA,-75.654907,45.350143
4,204,AIRPORT PKWY,UPLANDS DR RAMP 62,NA,-75.651611,45.331390


In [36]:
hour_df.head()

,Hour_Key,Hour_Start,Hour_End,Date,Month,Year
0,300,19,20,13/1/2017,1,2017
1,301,12,13,15/3/2017,3,2017
2,302,18,19,19/4/2017,4,2017
3,303,13,14,21/12/2017,12,2017
4,304,8,9,30/10/2017,10,2017


In [37]:
collision_df.to_csv('../Data/accident_dimension.csv', index=False)
location_df.to_csv('../Data/location_dimension.csv', index=False)
hour_df.to_csv('../Data/hour_dimension.csv', index=False)